
#  the model in this file comes from this website

#  https://geertlitjens.nl/post/getting-started-with-camelyon/



#  without editing the model, it is 80% accurate/confident

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import tensorflow_datasets as tfds

pcam, pcam_info = tfds.load('patch_camelyon', shuffle_files=True, with_info=True,
                            data_dir='/content/drive/My Drive/Colab Notebooks')

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

Shuffling and writing examples to /content/drive/My Drive/Colab Notebooks/patch_camelyon/2.0.0.incompleteURT6UC/patch_camelyon-test.tfrecord


Shuffling and writing examples to /content/drive/My Drive/Colab Notebooks/patch_camelyon/2.0.0.incompleteURT6UC/patch_camelyon-train.tfrecord


Shuffling and writing examples to /content/drive/My Drive/Colab Notebooks/patch_camelyon/2.0.0.incompleteURT6UC/patch_camelyon-validation.tfrecord


Dataset patch_camelyon downloaded and prepared to /content/drive/My Drive/Colab Notebooks/patch_camelyon/2.0.0. Subsequent calls will reuse this data.


In [0]:
# Import NumPy to handle array's and Matplotlib for plotting loss curves
import numpy as np
import matplotlib.pyplot as plt

# Import TensorFlow and relevant Keras classes to setup the model
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, Activation, LayerNormalization, Cropping2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
#just some network parameters, see above link regarding the layers for details
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

#dropout is used for regularization here with a probability of 0.3 for conv layers, 0.5 for the dense layer at the end
dropout_conv = 0.3
dropout_dense = 0.5

#initialize the model
model = Sequential()

#now add layers to it

#conv block 1
model.add(Conv2D(first_filters, kernel_size, input_shape = (96, 96, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(first_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))



#conv block 2
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))


#conv block 3
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))


#a fully connected (also called dense) layer at the end
model.add(Flatten())
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_dense))

model.add(Dense(2, activation = "softmax"))

model.compile(optimizer=Adam(0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])
# model.summary()

In [0]:
import random

def convert_sample(sample):
    """
    Preprocesses images by random rotation
    and greyscaling
    """
    image, label = sample['image'], sample['label']  
    image = tf.image.convert_image_dtype(image, tf.float32)

    ### Begin Preprocessing ###

    # Random rotation, reflection of the image
    image = tf.image.rot90(image, random.randint(0,3))
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    # Image conversion to grayscale
    image = tf.image.rgb_to_grayscale(image)

    # Image cropping
    # Label is 32x32(?)
    # central_region = 0.5 # 48x48
    # image = tf.image.central_crop(image, central_region)
    # image = tf.image.resize(image, (96, 96))
    
    ### End Preprocessing ###

    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label

In [0]:
train_pipeline = pcam['train'].map(convert_sample, num_parallel_calls=8).shuffle(1024).repeat().batch(64).prefetch(2)
valid_pipeline = pcam['validation'].map(convert_sample, num_parallel_calls=8).repeat().batch(128).prefetch(2)

In [0]:
hist = model.fit(train_pipeline,
                 validation_data=valid_pipeline,
                 verbose=2, epochs=10, steps_per_epoch=256, validation_steps=256)

Epoch 1/10
256/256 - 31s - loss: 0.5729 - accuracy: 0.7443 - val_loss: 1.8728 - val_accuracy: 0.5091
Epoch 2/10
256/256 - 31s - loss: 0.4948 - accuracy: 0.7690 - val_loss: 1.3952 - val_accuracy: 0.6059
Epoch 3/10
256/256 - 30s - loss: 0.4678 - accuracy: 0.7896 - val_loss: 0.4635 - val_accuracy: 0.7820
Epoch 4/10
256/256 - 27s - loss: 0.4632 - accuracy: 0.7874 - val_loss: 0.4606 - val_accuracy: 0.7954
Epoch 5/10
256/256 - 28s - loss: 0.4513 - accuracy: 0.7961 - val_loss: 0.4991 - val_accuracy: 0.7832
Epoch 6/10
256/256 - 30s - loss: 0.4425 - accuracy: 0.8034 - val_loss: 0.5112 - val_accuracy: 0.8021
Epoch 7/10
256/256 - 30s - loss: 0.4372 - accuracy: 0.8045 - val_loss: 0.4274 - val_accuracy: 0.8024
Epoch 8/10
256/256 - 30s - loss: 0.4186 - accuracy: 0.8198 - val_loss: 0.4847 - val_accuracy: 0.8012
Epoch 9/10
256/256 - 30s - loss: 0.4106 - accuracy: 0.8188 - val_loss: 0.3751 - val_accuracy: 0.8270
Epoch 10/10
256/256 - 30s - loss: 0.3906 - accuracy: 0.8278 - val_loss: 0.3566 - val_accura

In [0]:
test_pipeline = pcam['test'].map(convert_sample, num_parallel_calls=8).batch(128).prefetch(2)
print("Test set accuracy is {0:.4f}".format(model.evaluate(test_pipeline, steps=128, verbose=0)[1]))

Test set accuracy is 0.8271


In [0]:
#model.save("./patchcamelyon.hf5")